In [ ]:
from spectra_visualization_helpers import * 
from scale_factor_helpers import *
import random
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
n_spectra_each_box = 10000

gps_list = [i for i in range(50)]
base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite"
gp_paths = [base_path+f"/gridpoint{i}/" for i in gps_list]
spectrum_num = random.sample(range(0, 10000), n_spectra_each_box)

wavelengths, fluxes, params = load_spectra_from_boxes(gp_paths, spectrum_num)

Ly_alpha_0 = 1215.67
boxsize = 25
z_box = 2
Ly_alphas_z = []
correct_redshifts = []

for i in range(len(wavelengths)):
    # wavelength_current = wavelengths[i]
    (Omega_m, Omega_b, Omega_Lambda, h) = params[i]
    corr_z = corrected_z(boxsize, z_box, Omega_m, Omega_Lambda, h)
    correct_redshifts.append(corr_z)
    Ly_alphas_z.append(redshift_wavelength_forward(corr_z, Ly_alpha_0))

In [ ]:
box_num = 0

############################################## spectra ##############################################

data = fluxes[box_num].tolist()
x = wavelengths[box_num]

print(len(data))
print(len(x))
left_cutoff = np.where(x > 3630)[0][0]
right_cutoff = np.where(x < 3700)[0][-1]

print(left_cutoff, right_cutoff)

for i in range(len(data)):
    data[i] = data[i][left_cutoff:right_cutoff]

x = x[left_cutoff:right_cutoff]

mins = [x[np.where(i == np.min(i))[0][0]] for i in data]
#mins = [np.mean(i) for i in data]
#mins = spectrum_num

print(mins)

_ , data = zip(*sorted(zip(mins, data), key=lambda x: x[0], reverse=True))

############################################## hist ##############################################

path = gp_paths[box_num] + "output/"
snapN = 3  # this is z=2
(Omega_m, Omega_b, Omega_Lambda, h) = params[box_num]
z_box = 2
Ly_alpha_0 = 1215.67

hist_vals, edges = custom_hist_z(path, snapN, n_bins=100)

lambda_edges = []
for edge in edges:
    corrected_z_edge = corrected_z(edge*1e-3, z_box, Omega_m, Omega_Lambda, h)
    converted_edge = redshift_wavelength_forward(corrected_z_edge, Ly_alpha_0)
    lambda_edges.append(converted_edge)

lambda_edges = np.array(lambda_edges)
# redshift_wavelength_forward(corrected_z_this_snap, edges)

centers = 0.5 * (lambda_edges[:-1] + lambda_edges[1:])


In [ ]:
plt.imshow(data, aspect='auto', origin='lower',
           extent=[x.min(), x.max(), 0, len(data)],
           cmap='magma')

# plt.plot([Ly_alphas_z[0], Ly_alphas_z[0]], [0, 1000])

plt.xlabel(r"Observed Wavelength [$\AA$]")
plt.ylabel("# of Spectrum")
plt.colorbar(label="Relaitve Flux")
plt.savefig("plots/2D_spectra_one_box.pdf", format="PDF")
plt.show()

In [ ]:
plt.plot(centers, hist_vals, drawstyle='steps-mid')
plt.xlabel(r"Observed Wavelength [$\AA$]")
plt.ylabel("Total gas mass in bin")
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(8,10))

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(2, 2, height_ratios=[0.2, 1], width_ratios=[1, 0.05], hspace=0., wspace=0)

ax_spect = fig.add_subplot(gs[1, 0])
ax_hist = fig.add_subplot(gs[0, 0], sharex=ax_spect)
cbar_ax = fig.add_subplot(gs[1, 1])

ax_hist.plot(centers, hist_vals, drawstyle='steps-mid')
ax_hist.tick_params(axis='x', direction='in')
ax_hist.tick_params(labelbottom=False)
ax_hist.tick_params(labelleft=False)

img_spect = ax_spect.imshow(data, aspect='auto', origin='lower',
                            extent=[x.min(), x.max(), 0, len(data)],
                            cmap='magma')

# plt.plot([Ly_alphas_z[0], Ly_alphas_z[0]], [0, 1000])

ax_spect.set_xlabel(r"Observed Wavelength [$\AA$]")
ax_spect.set_ylabel("# of Spectrum")

###################################### dashed line #####################################

hist_peak_posi = centers[np.where(hist_vals == hist_vals.max())]
ax_hist.plot([hist_peak_posi, hist_peak_posi], [hist_vals.min(), hist_vals.max()], c="black", linestyle="--", alpha=0.3)
ax_spect.plot([hist_peak_posi, hist_peak_posi], [0, n_spectra_each_box], c="black", linestyle="--", alpha=0.3)

########################################## ##########################################
cbar = fig.colorbar(img_spect, cax=cbar_ax, orientation='vertical')
cbar.set_label(r'Relative Flux')

plt.savefig("plots/2D_spectra_one_box_with_hist.pdf", format="PDF")
plt.show()

In [ ]:
spectrum_N = 600
data = []
for sp in fluxes[:]:
    data.append(sp[spectrum_N])
print(data)
x = wavelengths[0]

print(len(data))
print(len(x))
left_cutoff = np.where(x > 3630)[0][0]
right_cutoff = np.where(x < 3700)[0][-1]

print(left_cutoff, right_cutoff)

for i in range(len(data)):
    data[i] = data[i][left_cutoff:right_cutoff]

x = x[left_cutoff:right_cutoff]

cosmo_par_index = 2  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
cosmo_par = [i[cosmo_par_index] for i in params]

_ , data = zip(*sorted(zip(cosmo_par, data), key=lambda x: x[0], reverse=False))
cosmo_par_sorted , Ly_alphas_sorted = zip(*sorted(zip(cosmo_par, Ly_alphas_z), key=lambda x: x[0], reverse=False))

In [ ]:
plt.imshow(data, aspect='auto', origin='lower',
           extent=[x.min(), x.max(), min(cosmo_par), max(cosmo_par)],
           cmap='magma')

plt.plot(Ly_alphas_sorted, cosmo_par_sorted, linestyle="--", color="black", label=r"$Ly \alpha$")
plt.legend(frameon=False)

plt.xlabel(r"Observed Wavelength [$\AA$]")
plt.ylabel(r"$\Omega_\Lambda$")
plt.colorbar(label="Relaitve Flux")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))#,dpi=500)

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(4, 5, width_ratios=[1, 1, 1, 1, 0.05], hspace=0, wspace=0)
axes = []

spectrum_numbers_to_use = random.sample(range(0, 1000), 4)*4

print(spectrum_numbers_to_use)

for j, spectrum_N in enumerate(spectrum_numbers_to_use): 

    # print(j, spectrum_N)
    
    row = j//4
    col = j%4
    
    ax = fig.add_subplot(gs[row, col])

    data = []
    for sp in fluxes[:]:
        data.append(sp[spectrum_N])

    x = wavelengths[0]

    left_cutoff = np.where(x > 3630)[0][0]
    right_cutoff = np.where(x < 3700)[0][-1]

    for i in range(len(data)):
        data[i] = data[i][left_cutoff:right_cutoff]

    x = x[left_cutoff:right_cutoff]

    label_dir = {0: r"$\Omega_m$", 1: r"$\Omega_\text{b}$", 2: r"$\Omega_\Lambda$", 3: r"h"}
    cosmo_par_index = row #2  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
    cosmo_par = [i[cosmo_par_index] for i in params]

    _ , data = zip(*sorted(zip(cosmo_par, data), key=lambda x: x[0], reverse=False))
    cosmo_par_sorted , Ly_alphas_sorted = zip(*sorted(zip(cosmo_par, Ly_alphas_z), key=lambda x: x[0], reverse=False))

    img_sp = ax.imshow(data, aspect='auto', origin='lower',
           extent=[x.min(), x.max(), min(cosmo_par), max(cosmo_par)],
           cmap='magma')
    
    ax.plot(Ly_alphas_sorted, cosmo_par_sorted, linestyle="--", color="black", label=r"$Ly \alpha$")
    ax.legend(loc="upper left", frameon=False)
    
    # fig.colorbar(img_sp, cax=ax, label="Relaitve Flux")

    if row == 3:
        ax.set_xlabel(r"Observed Wavelength [$\AA$]")
    else:
        ax.set_xticklabels([])
        ax.tick_params(axis='x', direction='in')

    if col == 0:
        ax.set_ylabel(f"{label_dir[row]}")
    else:
        ax.set_yticklabels([])
    # ax.set_xlim([3630, 3710])
    
    # ax.set_title(f"Spectrum #{spectrum_N}")
    # ax.set_facecolor('black')
    axes.append(ax)


cbar_ax = fig.add_subplot(gs[:, 4])
cbar = fig.colorbar(img_sp, cax=cbar_ax, orientation='vertical')
cbar.set_label(r'Relative Flux')
# plt.tight_layout()
plt.savefig("plots/2D_spectra_grid_4_params.pdf", format="PDF")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))#,dpi=500)

n_rows = 6
n_cols = 5

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(n_rows, n_cols+1, width_ratios=[1]*n_cols +[0.05], hspace=0, wspace=0)
axes = []

spectrum_numbers_to_use = random.sample(range(0, 1000), n_rows*n_cols)

print(spectrum_numbers_to_use)

for j, spectrum_N in enumerate(spectrum_numbers_to_use): 

    # print(j, spectrum_N)
    divisor = min(n_cols, n_cols)
    row = j//divisor
    col = j%divisor

    
    ax = fig.add_subplot(gs[row, col])

    data = []
    for sp in fluxes[:]:
        data.append(sp[spectrum_N])

    x = wavelengths[0]

    left_cutoff = np.where(x > 3630)[0][0]
    right_cutoff = np.where(x < 3700)[0][-1]

    for i in range(len(data)):
        data[i] = data[i][left_cutoff:right_cutoff]

    x = x[left_cutoff:right_cutoff]

    label_dir = {0: r"$\Omega_m$", 1: r"$\Omega_\text{b}$", 2: r"$\Omega_\Lambda$", 3: r"h"}
    cosmo_par_index = 2  # 0: Omega0, 1: OmegaBaryon, 2: OmegaLambda, 3: HubbleParam
    cosmo_par = [i[cosmo_par_index] for i in params]

    _ , data = zip(*sorted(zip(cosmo_par, data), key=lambda x: x[0], reverse=False))
    cosmo_par_sorted , Ly_alphas_sorted = zip(*sorted(zip(cosmo_par, Ly_alphas_z), key=lambda x: x[0], reverse=False))

    img_sp = ax.imshow(data, aspect='auto', origin='lower',
           extent=[x.min(), x.max(), min(cosmo_par), max(cosmo_par)],
           cmap='magma')
    
    ax.plot(Ly_alphas_sorted, cosmo_par_sorted, linestyle="--", color="black", alpha=0.6)
    ax.annotate(r"$Ly \alpha$", 
                [Ly_alphas_sorted[-round(len(Ly_alphas_sorted)/10)]+5, cosmo_par_sorted[-round(len(Ly_alphas_sorted)/10)]],
                alpha=0.6)
    
    # fig.colorbar(img_sp, cax=ax, label="Relaitve Flux")
    # ax.annotate(f"# {spectrum_N}", [x.min() + (x.max()-x.min())/30, max(cosmo_par)-(max(cosmo_par)-min(cosmo_par))/15], alpha=0.5, c="dimgray")

    if row !=n_rows-1:
        ax.set_xticklabels([])
        ax.tick_params(axis='x', direction='in')

    if col != 0:
        ax.set_yticklabels([])
    # ax.set_xlim([3630, 3710])
    
    # ax.set_title(f"Spectrum #{spectrum_N}")
    # ax.set_facecolor('black')
    axes.append(ax)

fig.supxlabel('Observed Wavelength [$\AA$]')
fig.supylabel(r"$\Omega_\Lambda$")
cbar_ax = fig.add_subplot(gs[:, n_cols])
cbar = fig.colorbar(img_sp, cax=cbar_ax, orientation='vertical')
cbar.set_label(r'Relative Flux')
plt.tight_layout()
plt.savefig("plots/2D_spectra_grid_1_param.pdf", format="PDF")
plt.show()